In [1]:
import SimpleITK as sitk 
import numpy as np 

path_pet = r'C:\Users\wendy\Documents\STAGE\PET0\1.2.250.1.74.20120625133000.1000031365770_nifti_PT.nii'
path_mask= r'C:\Users\wendy\Documents\STAGE\PET0\1.2.250.1.74.20120625133000.1000031365770_nifti_mask_3d.nii'
path_ct = r'C:\Users\wendy\Documents\STAGE\PET0\1.2.250.1.74.20120625133000.1000031365770_nifti_CT.nii'

ct_img = sitk.ReadImage(path_ct)
print(ct_img.GetSize())
pet_img = sitk.ReadImage(path_pet)
mask_img = sitk.ReadImage(path_mask)

#ct = r'C:\Users\wendy\Documents\STAGE\PET0\image_9d1f6b1606e0b318a8dff0928564ccc4.nii'
#ct_img = sitk.ReadImage(ct)
#print(ct_img.GetSize())

(512, 512, 287)


In [3]:
from dicom_to_cnn.model.reader.Nifti import Nifti 
from dicom_to_cnn.model.post_processing.mip.MIP_Generator import MIP_Generator 

objet = Nifti(path_ct)
resampled = objet.resample(shape=(256, 256, 1024))
resampled[np.where(resampled < 500)] = 0 #500 UH
normalize = resampled[:,:,:,]/np.max(resampled)
mip_generator = MIP_Generator(normalize)
mip_generator.project(angle=0)
mip_generator.save_as_png('mip_ct',  r'C:\Users\wendy\Documents\STAGE\PET0', vmin=0, vmax=1)


256
256
480
(1024, 256, 256)


'C:\\Users\\wendy\\Documents\\STAGE\\PET0\\mip_ct.png'

In [21]:
from skimage import filters
def roi_seg(mask_array:np.ndarray, pet_array:np.ndarray, threshold:str='0.41'): 
    """otsu segmentation of mask_array np.ndarray

        Args:
            mask_array (np.ndarray): [shape [c,z,y,x]]
            pet_array (np.ndarray): [shape [z,y,x]]
            threshold (str): ['otsu', '0.41', '2.5', and '4.0']

        Returns:
            [np.ndarray]: [shape [z,y,x]]
    """
    new_mask = np.zeros(mask_array.shape[1:], dtype=np.uint8)
    for num_slice in range(mask_array.shape[0]):
        mask_slice = mask_array[num_slice]  # R.O.I
        roi = pet_array[mask_slice > 0]
        if len(roi) == 0:
            continue
        try:
                        # apply threshold
            if threshold == 'otsu' : 
                t = filters.threshold_otsu(roi)
            if threshold == '0.41' : 
                t = np.max(roi) * float(threshold)
            if threshold == '2.5' or threshold == '4.0' : 
                t = float(threshold)
            new_mask[np.where((pet_array >= t) & (mask_slice > 0))] = 1
        except Exception as e:
            print(e)
            print(sys.exc_info()[0])
    return new_mask


def roi2mask(mask_img, pet_img):
    """
    Generate the mask from the ROI of the pet scan
    Args:
            :param mask_img: sitk image, raw mask (i.e ROI)
            :param pet_img: sitk image, the corresponding pet scan

    :return: sitk image, the ground truth segmentation
    """
        # transform to numpy
    origin = mask_img.GetOrigin()
    spacing = mask_img.GetSpacing()
    direction = tuple(mask_img.GetDirection())
    mask_array = sitk.GetArrayFromImage(mask_img)
    pet_array = sitk.GetArrayFromImage(pet_img)

        # get 3D meta information
    if len(mask_array.shape) == 3:
        mask_array = np.expand_dims(mask_array, axis=0)
    else:
        mask_array = np.transpose(mask_array, (3,0,1,2))

    new_masks = []
        #otsu 
    new_masks.append(roi_seg(mask_array, pet_array, threshold='otsu'))
        #41%
    new_masks.append(roi_seg(mask_array, pet_array, threshold='0.41'))
        #2.5
    new_masks.append(roi_seg(mask_array, pet_array, threshold='2.5'))
        #4.0
    new_masks.append(roi_seg(mask_array, pet_array, threshold='4.0'))

    new_mask = new_masks[0] if len(new_masks) == 1 else np.mean(np.array(new_masks), axis=0)

                # reconvert to sitk and restore 3D meta-information
    print(new_mask.shape)
    print(np.unique(new_mask))
    new_mask = sitk.GetImageFromArray(new_mask)
    new_mask.SetOrigin(origin)
    new_mask.SetDirection(direction)
    new_mask.SetSpacing(spacing)

    return new_mask

In [22]:
matrix = roi2mask(mask_img, pet_img)

(287, 128, 128)
[0.   0.25 0.5  0.75 1.  ]


In [17]:
matrix.GetSize()

(128, 128, 287)

In [2]:
print(ct_img.GetSize())
print(pet_img.GetSize())
print(mask_img.GetSize())

(512, 512, 287)
(128, 128, 287)
(128, 128, 287)


In [6]:
target_size = (128, 128, 256)
target_spacing = (4.0, 4.0, 4.0)
target_direction = (1,0,0,0,1,0,0,0,1)
objet = Fusion()
objet.set_origin_image(pet_img)
objet.set_target_volume(target_size, target_spacing, target_direction)
resampled_pet_img = objet.resample(pet_img, defaultValuePixel = -1000.0)
print(resampled_pet_img.GetSize())

(128, 128, 256)


In [1]:
from dicom_to_cnn.model.segmentation.MaskBuilder_DICOMSEG import MaskBuilder_DICOMSEG 

path =  r'C:\Users\wendy\Documents\STAGE\PET0\sarcome_dicom_seg.dcm'

objet = MaskBuilder_DICOMSEG(path)
matrix = objet.get_numpy_array()

In [2]:
matrix.shape

(139, 256, 256)